In [ ]:
!gdown 1fCerWKN3Jd9cGGu3n76hQDxrP744RTBi

In [ ]:
import numpy
import pandas as pd
import nltk
from pandas import DataFrame
from string import punctuation
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
data = pd.read_csv("Train Final.csv")
print(data.size/2)

In [ ]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in punctuation])
    return punctuationfree
#storing the puntuation free text
data['lower']= data['email'].apply(lambda x:remove_punctuation(x))

data['clean_msg']= data['lower'].apply(lambda x: x.lower())

import re
def tokenization(text):
    tokens = re.split('\W+',text)
    return tokens
#applying function to the column
data['token']= data['clean_msg'].apply(lambda x: tokenization(x))


import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

data['stop']= data['token'].apply(lambda x:remove_stopwords(x))


#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

data['stem']=data['stop'].apply(lambda x: stemming(x))


from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
data['lemma']=data['stem'].apply(lambda x:lemmatizer(x))

# from nltk.corpus import words
# nltk.download('words')
# corpus = set(words.words())
# def remove_not_words(text):
#     realwords = [i for i in text if i in corpus]
#     return realwords

# data['final']= data['lemma'].apply(lambda x:remove_not_words(x))
data['final']= data['lemma']

def wordtobin(verdict):
    if verdict == 'ham': 
        return 0
    return 1

data['verdict'] = data['verdict'].apply(lambda x: wordtobin(x))

data.head()


## **ELMo**

In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

In [ ]:
# import necessary libraries
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# Load pre trained ELMo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

# create an instance of ELMo
embeddings = elmo(
	[
		"I love to watch TV",
		"I am wearing a wrist watch"
	],
	signature="default",
	as_dict=True)["elmo"]
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

# Print word embeddings for word WATCH in given two sentences
print('Word embeddings for word WATCH in first sentence')
print(sess.run(embeddings[0][3]))
print('Word embeddings for word WATCH in second sentence')
print(sess.run(embeddings[1][5]))


In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

embed = []

sample = data['final'].sample(10)
for i in sample:
  embed.append(elmo_vectors(i))

print(embed)

## **Word2Vec Embeddings**

In [ ]:
## word2vec embedding 
import gensim
from gensim.models import Word2Vec
input=[]
for i in data['final']:  
  input.append(i)
print(input[:10])
model1 = gensim.models.Word2Vec(input, min_count = 1,size = 200, window = 10)
print(model1)
# print(model1)

In [ ]:
my_dict = dict({})
for idx, key in enumerate(model1.wv.vocab):
  my_dict[key] = model1.wv[key]

In [ ]:
import numpy as np
vector=[]
for i in data['final']:
  p = 0
  ve=[0.0]
  for j in i:
    p=p+1
    ve=ve+my_dict[j]
  np.divide(ve,p)
  vector.append(ve)
data['email_word2vec']=vector




## **RoBERTa Word Embeddings**

In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('all-distilroberta-v1')

In [ ]:
#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("\n")

In [ ]:
# sentences = ['This framework generates embeddings for each input sentence',
#     'Sentences are passed as a list of string.',
#     'The quick brown fox jumps over the lazy dog.']

# #Sentences are encoded by calling model.encode()
# embeddings = model.encode(sentences)

# #Print the embeddings
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("\n")
email_embedding=[]
for each_email in data['email']:
  # print(each_email)
  email_embedding.append(model.encode(each_email))

#print(email_embedding)
data['bert']=email_embedding


In [ ]:
print(data['bert'])

In [ ]:
print(email_embedding[0])
print(email_embedding[30])
print(email_embedding[100])

## **Named Entity Recognition (NER) Code**

In [ ]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [ ]:
spacy.explain("ORG")
spacy.explain("LOC")
spacy.explain("MONEY")
spacy.explain("GPE")
spacy.explain("DATE")

In [ ]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

In [ ]:
email_list=data['email'].tolist()

In [ ]:
print(email_list[9])
print(email_list[900])
print(email_list[3000])

In [ ]:
print(len(email_list))

In [ ]:
emailNER=[]
for each_email in email_list:
  emailNER.append(NER(each_email))
  display=displacy.render(emailNER[len(emailNER)-1],style="ent",jupyter=True)
  print(len(emailNER))
  print("\n\n")